In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
import datetime

current_date = datetime.datetime.now().date()

target_date = datetime.date(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

# LLMChain

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [4]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

/var/folders/y5/gsqj_t056ys0vzl4s8m110d80000gn/T/ipykernel_8729/1023897470.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.0, model=llm_model)


In [5]:
prompt = ChatPromptTemplate.from_template(
    "What is the best way to describe company that makes a product {product}"
)

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/y5/gsqj_t056ys0vzl4s8m110d80000gn/T/ipykernel_8729/1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [9]:
product = "Nvidia GPUs"
chain.run(product=product)

'Nvidia is a leading technology company that specializes in designing and manufacturing high-performance graphics processing units (GPUs) for a wide range of applications, including gaming, artificial intelligence, data centers, and autonomous vehicles.'